# Week 6 Exercises

_McKinney 6.1_

There are multiple ways to solve the problems below.  You can use any one of several approaches.  For example, you can read CSV files using Pandas or the csv module.  Your score won't depend on which modules you choose to use unless explicitly noted below, but your programming style will still matter.

### 30.1 List of Allergies

In this GitHub repository, there is a file called `allergies.json` that contains a list of patient allergies.  You will need to download this [file from here](https://raw.githubusercontent.com/paulboal/hds5210-2023/main/week06/allergies.json) and then upload it into Google Colab to run these examples. It is taken from sample data provided by the EHR vendor, Epic, here: https://open.epic.com/Clinical/Allergy

Take some time to look at the structure of the file.  You can open it directly in Jupyter by clicking the _Home_ icon, then the _from_instructor_ folder, and then the _data_ folder.

Within the file, you'll see that it is a dictionary with many items in it.  One of those items is called `entry` and that item is a list of things.  You can tell that because the item name is immediately followed by an opening square bracket, signifying the start of a list.  It's line 11 of the file: `  "entry": [`

Write a function named `allergy_count(json_file)` that takes as one parameter the name of the JSON file and returns an integer number of entries in that file.  Your function should open the file, read the json into a Python object, and return how many items there are in the list of `entry`s.

In [1]:
import json
ALLERGIES_FILE="allergies.json"

In [2]:
def allergy_count(json_file):
    """
    It Counts the number of allergy entries in a JSON file.

    Arguments:
        json_file (str): The path to the JSON file containing allergy data.

    Returns:
        int: The number of allergy entries in the JSON file.
    """
    with open(json_file) as f:
        data = json.load(f)
        return len(data['entry'])

In [3]:
allergy_count(ALLERGIES_FILE)

4

In [4]:
assert type(allergy_count(ALLERGIES_FILE)) == int
assert allergy_count(ALLERGIES_FILE) == 4

### 30.2 Number of Patients

If you dig a little bit deaper into this list of allergies, you'll see that each result has a patient associated with it.  Create a funcation called `patient_count(json_file)` that will count how many unique patients we have in this JSON structure.  

In [5]:
def patient_count(json_file):
    """
    Counts the number of unique patients with allergies in a JSON file.

    Arguments:
        json_file (str): The path to the JSON file containing allergy data.

    Returns:
        int: The number of unique patients with allergies in the JSON file.
    """
    with open(json_file) as f:
        data = json.load(f)
        return len({entry['resource']['patient']['display'] for entry in data['entry']})

In [6]:
patient_count(ALLERGIES_FILE)

2

In [7]:
assert type(patient_count(ALLERGIES_FILE)) == int
assert patient_count(ALLERGIES_FILE) == 2

### 30.3 How Many Allergies per Patient

Although each entry is a separate allergy, several of them are for the same patient.  Write a function called `allergy_per_patient(json_file)` that counts up how many allergies each patient has.


In [8]:
def allergy_per_patient(json_file):
    """
    Counts the number of allergies per patient in a JSON file.

    Argumens:
        json_file (str): The path to the JSON file containing allergy data.

    Returns:
        dict: A dictionary where keys are patient names and values are the count of allergies for each patient.
    """
    with open(json_file) as f:
        data = json.load(f)
        patient_allergies = {}
        for entry in data['entry']:
            patient = entry['resource']['patient']['display']
            if patient not in patient_allergies:
                patient_allergies[patient] = 0
            patient_allergies[patient] += 1
        return patient_allergies


In [9]:
allergy_per_patient(ALLERGIES_FILE)

{'Jason Argonaut': 3, 'Paul Boal': 1}

In [10]:
assert type(allergy_per_patient(ALLERGIES_FILE)) == dict
assert allergy_per_patient(ALLERGIES_FILE) == {'Paul Boal': 1, 'Jason Argonaut': 3}

### 30.4 Patient Allergies and Reaction

You'll see in the file that each of the items in the `entry` list have several other attributes including a patient name, substance text representation, and a reaction manifestation.  Create a function named `allergy_list(json_file)` that will create an output list that has patient name, allergy, and reaction for each `entry`.  The actual result you should get will be:

```python
[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]
```

You'll notice that the reaction and the manifestation of that action are lists.  You only need to capture the first reaction and the first manifestation of the action.  That is, if there is a list of things, just output the first one.

In [11]:
def allergy_list(json_file):
    """
    Create a list of allergies from a JSON file.

    Arguments:
        json_file (str): The path to the JSON file containing allergy data.

    Returns:
        list: A list of lists where each sublist contains patient name, substance, and reaction for each allergy entry.
    """
    with open(json_file) as f:
        data = json.load(f)
        allergies = []
        for entry in data['entry']:
            patient = entry['resource']['patient']['display']
            substance = entry['resource']['substance']['text']
            reaction = entry['resource']['reaction'][0]['manifestation'][0]['text']
            allergies.append([patient, substance, reaction])
        return allergies

In [12]:
allergy_list(ALLERGIES_FILE)

[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]

In [13]:
assert allergy_list(ALLERGIES_FILE) == [['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]


### 30.5 Allergy Reaction

Write a function called `allergy_reaction(json_file,patient,substance)` that takes three parameter and returns the reaction that will happen if the patient takes the specified substance.  You can solve this, in part, by calling your `allergy_list` function inside your new `allergy_reaction` function.

If the substance is not found in the allergy list, the function should return None.

In [14]:
def allergy_reaction(json_file, patient, substance):
    """
    Get the reaction for a specific allergy of a patient to a substance from a JSON file.

    Arguments:
        json_file (str): The path to the JSON file containing allergy data.
        patient (str): The name of the patient.
        substance (str): The name of the substance to check for allergy reaction.

    Returns:
        str or None: The allergy reaction if found, or None if not found.
    """
    allergies = allergy_list(json_file)
    for allergy in allergies:
        if allergy[0] == patient and allergy[1] == substance:
            return allergy[2]
    return None


In [15]:
allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G')

'Hives'

In [16]:
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') == 'Hives'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS') == 'Itching'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'STRAWBERRY') == 'Anaphylaxis'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN') == None
assert allergy_reaction(ALLERGIES_FILE, 'Paul Boal', 'PENICILLIN G') == 'Bruising'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to save this notebook file back to GitHub.  To do that in Google Colab:
1. File -> Save a Copy in GitHub
2. Make sure your HDS5210 repository is selected
3. Make sure the file name includes the week number like this: `week06/week06_assignment_2.ipynb`
4. Add a commit message that means something

**Be sure week names are lowercase and use a two digit week number!!**

**Be sure you use the same file name provided by the instructor!!**

